# Competitiveness of Taiwan Machine Tools (HS 8456-8466)

#### Part 1: Sum over All Machine Tools

In [1]:
import pandas as pd

path = '//172.26.1.102/dstore/uncomtrade/annual/type-C_r-ALL_ps-2015_freq-A_px-HS_pub-20170213_fmt-csv_ex-20170226.csv'
df = pd.read_csv(path, dtype={'Commodity Code': str})

In [2]:
mt_regex = '^' + '|^'.join(map(str, range(8456, 8467)))
mt = df.loc[(df['Aggregate Level'] == 6) & (df['Trade Flow'] == 'Import') &
            (df['Partner'] != 'World') & (df['Commodity Code'].str.contains(mt_regex)),
            ['Reporter', 'Partner', 'Commodity Code', 'Commodity', 'Trade Value (US$)']]
mt.columns = ['reporter', 'partner', 'code', 'commodity', 'val']

mt_sum = mt.groupby(['reporter', 'partner']).sum().reset_index()
# Total import of each reporter
mt_sum['world'] = mt_sum.groupby('reporter')['val'].transform('sum')
# Market share for each partner country
mt_sum['share'] = mt_sum['val'] / mt_sum['world'] * 100
# Rank for each partner country
mt_sum['rank'] = mt_sum.groupby('reporter')['val'].rank(ascending=False, method='min')

In [3]:
# World top 20 importer
top20 = mt_sum.groupby('reporter').agg({'val': 'sum'}).sort_values(
    'val', ascending=False).head(20).reset_index().rename(
    columns=dict(reporter='進口國', val='自全球進口額(美元)'))
top20['排名'] = range(1, len(top20) + 1)
top20 = top20[['排名', '進口國', '自全球進口額(美元)']]

In [4]:
import numpy as np

mt_sum = mt_sum[(mt_sum['partner'] == 'Other Asia, nes') & (mt_sum['rank'] <= 3)].sort_values(
    ['rank', 'val'], ascending=[True, False]).drop('partner', axis=1)[
    ['reporter', 'val', 'world', 'share', 'rank']]
mt_sum = mt_sum.assign(in_top20=mt_sum['reporter'].apply(
        lambda x: np.where(x in top20['進口國'].values, 'Y', 'N'))).rename(
    columns=dict(reporter='進口國', val='自臺進口額(美元)', world='自全球進口額(美元)',
                 share='臺灣市占率(%)', rank='臺灣排名', in_top20='為全球前20大進口國'))

#### Part 2: By HS6 Products

In [5]:
df = pd.read_csv('C:/Users/2093/Desktop/Data Center/09. 產品別出口市場/comp_aggregate_6_utf-8.csv',
                 dtype={'product': str})

In [6]:
eighteen = ['不丹', '印尼', '印度', '孟加拉', '寮國', '尼泊爾', '巴基斯坦', '斯里蘭卡', '新加坡',
            '柬埔寨', '汶萊', '泰國', '澳大利亞', '紐西蘭', '緬甸', '菲律賓', '越南', '馬來西亞']
mt_p = df[(df['country'].apply(lambda x: x in eighteen)) & (df['product'].str.contains(mt_regex)) &
          (df['tw_rank'] == 1)]

# For each category of machine tools, compute no. of importing countries where Taiwan ranks #1
n_country = mt_p.groupby('product').agg({'country': 'count'}).reset_index()

# Build a list of countries and their market share pasted together
n_country['countries'] = [', '.join(pd.Series(list(zip(
                    mt_p.loc[mt_p['product'] == p, 'country'].values,
                    mt_p.loc[mt_p['product'] == p, 'tw_share'].apply(
                        lambda x: '(' + str(round(x, 2)) + '%)')))).apply(
            lambda x: ''.join(x)).values) for p in n_country['product']]

n_country = n_country.sort_values('product').merge(
    mt_p[['product', 'desc2', 'desc4', 'desc6']].drop_duplicates(), on='product')[
    ['product', 'desc2', 'desc4', 'desc6', 'country', 'countries']].rename(
    columns=dict(product='產品代碼', desc2='HS2名稱', desc4='HS4名稱', desc6='HS6名稱',
                 country='臺灣排名第1進口國數', countries='國家(臺灣市占率)'))

In [7]:
from pandas import ExcelWriter

# Write to separate worksheets
writer = ExcelWriter('machine_tool.xlsx')
mt_sum.to_excel(writer, sheet_name='(總和)臺灣排名前3進口國', index=False)
top20.to_excel(writer, sheet_name='(總和)全球前20大進口國', index=False)
n_country.to_excel(writer, sheet_name='(HS6產品別)臺灣排名第1進口國', index=False)
writer.save()